<a href="https://colab.research.google.com/github/anacasicande/Modelamiento/blob/main/Destilacio%CC%81n1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import numpy as np
import matplotlib.pyplot as plt

def calcular_constantes_equilibrio(temperaturas, alpha, beta, gamma):
    # Calcular las constantes de equilibrio K_ji para cada componente en cada etapa
    K = np.zeros((len(temperaturas), len(alpha)))
    for j, T in enumerate(temperaturas):
        for i in range(len(alpha)):
            K[j, i] = alpha[i] + beta[i] * T + gamma[i] * T**2
    return K

def resolver_sistema_lineal_rectificacion(num_etapas, A0, razones_adsorcion):
    # Inicializar la matriz A y el vector b para Ax = b en la zona de rectificación
    A = np.zeros((num_etapas, num_etapas))
    b = np.ones(num_etapas)
    b[0] = A0 + 1  # Establecer el valor para la primera ecuación (condensador)

    # Llenar la matriz bidiagonal A para la zona de rectificación
    for j in range(num_etapas):
        if j == 0:
            A[j, j] = 1  # Primer elemento diagonal para el condensador
        else:
            A[j, j] = 1  # Elemento diagonal
            A[j, j-1] = -razones_adsorcion[j-1]  # Elemento subdiagonal

    # Resolver el sistema lineal
    x = np.linalg.solve(A, b)

    return x

def resolver_sistema_lineal_stripping(num_etapas, A0, razones_adsorcion):
    # Inicializar la matriz A y el vector b para Ax = b en la zona de stripping
    A = np.zeros((num_etapas, num_etapas))
    b = np.ones(num_etapas)
    b[0] = A0 + 1  # Establecer el valor para la primera ecuación (rehervidor)

    # Llenar la matriz bidiagonal A para la zona de stripping
    for j in range(num_etapas):
        if j == 0:
            A[j, j] = 1  # Primer elemento diagonal para el rehervidor
        else:
            A[j, j] = 1  # Elemento diagonal
            A[j, j-1] = -razones_adsorcion[j-1]  # Elemento subdiagonal

    # Resolver el sistema lineal
    x = np.linalg.solve(A, b)

    return x

# Datos dados
F = 1000  # mol/h (Caudal de alimentación)
z = [0.06, 0.17, 0.22, 0.20, 0.35]  # Fracciones molares de alimentación para los componentes 1 a 5
relacion_reflujo = 2.5  # L1 / D
temperaturas_F = [140, 150, 160, 170, 180, 190, 200, 210, 220, 230]  # Temperaturas en °F
temperaturas_K = [(T - 32) * 5/9 + 273.15 for T in temperaturas_F]  # Convertir a Kelvin para consistencia

# Parámetros de las constantes de equilibrio para cada componente (de la Tabla P2.5)
alpha = [0.70, 2.21, 1.50, 0.86, 0.71]
beta = [0.30e-2, 1.95e-2, -1.60e-2, -0.97e-2, -0.87e-2]
gamma = [0.65e-4, 0.90e-4, 0.80e-4, 0.46e-4, 0.42e-4]

# Paso 1: Calcular las constantes de equilibrio
K = calcular_constantes_equilibrio(temperaturas_F, alpha, beta, gamma)

#Sacar L y V
D=500
L_D=2.5
L=D*L_D
V=D+L

# Paso 2: Calcular los caudales de rectificación y stripping

# Definir los parámetros de las zonas de rectificación y stripping
num_etapas_rectificacion = 5  # Número de etapas en la zona de rectificación
num_etapas_stripping = 5  # Número de etapas en la zona de stripping
A0_rectificacion = L/D  # Valor inicial para la razón de adsorción en el condensador
A0_stripping = L/D  # Valor inicial para la razón de adsorción en el rehervidor
razones_adsorcion_rectificacion = [0.5, 0.6, 0.7, 0.8]  # Ejemplo de razones de adsorción para la rectificación
razones_adsorcion_stripping = [0.7, 0.8, 0.9, 1.0]  # Ejemplo de razones de adsorción para la stripping

# Resolver el sistema lineal para la zona de rectificación
caudales_rectificacion = resolver_sistema_lineal_rectificacion(num_etapas_rectificacion, A0_rectificacion, razones_adsorcion_rectificacion)

# Resolver el sistema lineal para la zona de stripping
caudales_stripping = resolver_sistema_lineal_stripping(num_etapas_stripping, A0_stripping, razones_adsorcion_stripping)

def valores_a(K):
# Hacemos una lista para almacenar los valores de a
    a_values = np.zeros_like(K)
# Calculamos los valores de a para cada Kij
    for i in range(K.shape[0]):  # Itera sobre las filas
       for j in range(K.shape[1]):  # Itera sobre las columnas
           if K[i, j]==0:
              a_values[i,j]=L_D

           else:
              a_values[i, j] = (L / (V * K[i, j]))
    return a_values
# Mostramos los resultados
P=valores_a(K)
print("\nValores de a:")
print(P)


# Mostrar los resultados numéricos
print("\nConstantes de equilibrio K_ji por etapa y componente:")
print(K)

print("\nCaudales de rectificación (v_ji / d_i) para cada etapa:")
for i, caudal in enumerate(caudales_rectificacion, start=1):
    print(f"Etapa {i}: {caudal:.4f}")

print("\nCaudales de stripping (v_ji / d_i) para cada etapa:")
for i, caudal in enumerate(caudales_stripping, start=1):
    print(f"Etapa {i}: {caudal:.4f}")

# Graficar los resultados de los caudales de stripping y rectificación
etapas_rectificacion = np.arange(1, num_etapas_rectificacion + 1)
etapas_stripping = np.arange(1, num_etapas_stripping + 1)




Valores de a:
[[0.29836496 0.1065462  0.86266391 1.76978621 2.26613488]
 [0.2734108  0.09976057 0.79365079 1.62337662 2.04081633]
 [0.25115531 0.09356638 0.72296125 1.47093434 1.81659642]
 [0.23127269 0.08790127 0.6541078  1.32177223 1.60585817]
 [0.21347451 0.08271025 0.58934465 1.18180959 1.41498755]
 [0.19750746 0.07794475 0.52988554 1.05414066 1.24613698]
 [0.18315018 0.07356187 0.47619048 0.93984962 1.0989011 ]
 [0.17020987 0.06952362 0.42822885 0.8387573  0.97155293]
 [0.1585188  0.0657964  0.38568343 0.749985   0.86183122]
 [0.14793118 0.06235036 0.34809245 0.67233219 0.76738904]]

Constantes de equilibrio K_ji por etapa y componente:
[[ 2.394   6.704   0.828   0.4036  0.3152]
 [ 2.6125  7.16    0.9     0.44    0.35  ]
 [ 2.844   7.634   0.988   0.4856  0.3932]
 [ 3.0885  8.126   1.092   0.5404  0.4448]
 [ 3.346   8.636   1.212   0.6044  0.5048]
 [ 3.6165  9.164   1.348   0.6776  0.5732]
 [ 3.9     9.71    1.5     0.76    0.65  ]
 [ 4.1965 10.274   1.668   0.8516  0.7352]
 [ 4.